# Examples on how to use the code for the matrix profile

In [ ]:
from src.configurations import Configuration
from src.continuous_series import ContinuousSeries
from src.helper import device_status_file_path_for
from src.matrix_profile import MatrixProfile
from src.read import read_flat_device_status_df_from_file
from src.stats import Sampling

## Read data

In [ ]:
zip_id = '' # set to a valid zip id
file_path = device_status_file_path_for(Configuration().perid_data_folder, zip_id)
full_df = read_flat_device_status_df_from_file(file_path, Configuration())

## Resample data

In [ ]:
max_interval = 180  # how frequent readings need per day, 60=every hour, 180=every three hours
min_days_of_data = 30  # how many days of consecutive readings with at least a reading every interval
time_col = Sampling.time_col # change this if you want to use another time column in the df
value_col = Sampling.iob_col # change this if you want to read another colum in the df
sample_rule = '1D' # see pandas doc for this, here 1 reading per day

# this is an array of all the continuous series available for the configurations above
series = ContinuousSeries(full_df, min_days_of_data, max_interval, time_col, value_col, sample_rule)

In [ ]:
# to get some details run
series.describe()

## Matrix Profile

In [ ]:
m = 7  # m window size, 7 days = weekly motives
resample_col = 'mean'  # which of the aggregated resample columns to take, options std, min, max, mean
cont_series = -1 #index to ContinuousSeries, -1 takes the last in the array, see describe outcome for more information
times, values = series.get_resampled_x_and_y_for(cont_series, resample_col)
matrix_profile = MatrixProfile(times, values, m)

In [ ]:
# plot motive at x
x = 0  # 0 = motif with the lowest distance (most like another bit of the ts)
matrix_profile.plot_ts_motif_and_profile(x, resample_col + ' ' + value_col, time_col, False)
matrix_profile.describe_motif_x(x) # prints distance and other useful information about the motive

In [ ]:
# find all motifs with a distance less than max_distance
max_distance = 2.33 # you can calculate the possible max distance, use that to set this number
min_neighbours = 1 # see stumpy for more details

#this just returns the motives
motif_distances, motive_indices = matrix_profile.top_motives(max_distance, min_neighbours)

#this plots them
matrix_profile.plot_top_motives_for_max_distance_and_min_neighbours(resample_col + ' ' + value_col, time_col,
                                                                    max_distance, min_neighbours, show_mp=True)

In [ ]:
# find the window that is least like any other sequence in the time series
x = matrix_profile.least_similar_x()
matrix_profile.plot_ts_motif_and_profile(x, resample_col + ' ' + value_col, time_col)
matrix_profile.describe_motif_x(x)

### Convenient methods

In [ ]:
# describes the series that went in
matrix_profile.describe_time_series()

In [ ]:
# to calculate what the maximum possible distance is for your m
matrix_profile.max_possible_distance()

In [ ]:
# to print information about the motiv at index x
matrix_profile.describe_motif_x(x)